In [2]:
const plantumlEncoder = require('plantuml-encoder')
const tslab = require("tslab");
import jwt_decode from "jwt-decode";

## Howto Share VC with Verifier
We already know about classical triangle of roles.
It is **user centric** 
- holder - user that own a VCs
- issuer - party that issue a VC
- verifier - could aks holder for a VCs to Verify

We need a **protocol** that could proof the identity and **autorship** for every role.
As we know our identities is our keypairs so we should sign some entities to proof our identity 
Affinidi SDK offer transport agnostic protocols for excanges based on JWT tokens

In [51]:

const encodedTriangle = plantumlEncoder.encode(`
@startuml
!include https://raw.githubusercontent.com/plantuml-stdlib/C4-PlantUML/master/C4_Container.puml

Person_Ext(issuer, "Issuer")
Boundary(b1,"Verification") {
Person(holder, "Holder", $sprite="person2")
Person(verifier, "Verifier")
}

Rel_L(issuer, holder, "issue VC", "Request for offer")
Rel(holder, issuer, "claim", "Response for offer")
Rel_R(verifier, holder, "ask for VCs", "Request for sharing (Presentation Challenge)")
Rel(holder, verifier, "issue signed VP", "VP", $sprite = "&envelope-closed")

@enduml
`)
 
const urlTriangle = 'http://www.plantuml.com/plantuml/svg/' + encodedTriangle
tslab.display.html(`<img src="${urlTriangle}"/>`)

## Flow

In [26]:
const encodedFlow = plantumlEncoder.encode(`
title: Affinidi: Share VC Flow
participant "Verifier Service" as verifier
actor  "Verifier client , sdk" as client
actor "SDK Holder" as holder
database "Wallet storage" as wallet
participant "Registry" as registry


client -> verifier: BuildCredentialRequest (credentialRequirements)
verifier -> client: 200 [credentialShareRequest]

note over client, verifier: credentialShareRequest signing and credentialShareRequestToken generated
client -> holder: some way deliver (credentialShareRequestToken)
note over client, verifier: deliver way could be as QR code which posted on web, as PN in internal App logic implementation, etc
note over client, verifier: Generating CredentialShareResponseToken based on the credentialShareRequestToken
note over client, verifier
Verification of the Verifier signature on the credentialShareRequestToken
1. Resolve Verifier DID (get publicKey)
2. Verify Verifier signature (using publicKey)
end note
holder -> registry: resolveDid(VerifierDid)
registry -> holder: 200 [DidDocument]
holder -> wallet: getCredentials(credentialShareRequestToken)
wallet -> holder: 200 [credentials]

holder -> holder: Generating VP with credentials inside
holder -> client: some way to call back (VP)

note over verifier, holder
Verifier validate credentials:
1. Resolve Holder DID (get publicKey)
2. Verify Holder signature on JWT (using publicKey)
3. Validate that Holder its a Subject in credentials(optional)
4. Resolve Issuer DID(get publicKey)
5. Verify Issuer signature on each credentials (using publicKey)

Executing internall App logic
end note
client -> registry: resolveDid(HolderDid)
registry -> client: 200 [DidDocument]
client -> registry: resolveDid(IssuerDid)
registry -> client: 200 [DidDocument]
client -> client : do VP Validation
client -> client : do VC Validation

`)
 
const urlFlow = 'http://www.plantuml.com/plantuml/svg/' + encodedFlow
tslab.display.html(`<img src="${urlFlow}"/>`)

<img src="http://www.plantuml.com/plantuml/svg/dLHDRzim3BthLn0vEK1MTDrsyQ50b-oZAp1aJP4UXXuKcR6vAfOdoSdoxoTPTYotQJeCy46MIBv7yf7CafMOmjLwJILbbCAYu1fXEONFKkrOnRKbGHKlBOoMg6bDg661Uai2Hy0DRDjBneLL6Y8h8GcTsmHCzjYOXelMSBIOtS1N9JFKpMFH75d6BLznWp2wvrAY1UECUHwmTitLaDGjvcIitZS6klrXZBNeRouF35FuM9FCfXepzq9StkBl6es5H0okIEF6_Panwpnzb10lXOlpS_ZPEpJbQWCzC5OgYw2sX-md4Vfn9p2KbrJcmCli1PCxzOWbv5YYvXQpABLGj1ICsg2hpXuobEJHanEHnYTPTX5yDA5gcS4AVMb_tBhVpA4K90geb75CG9MmmzN4ly-_0vNkiwXBBk6gga2gd0JGff9DGRabLKu0hJX9u4l8qvTa-ZG9KwdIOAY7Lqd3m1Pugd0dm5ZGgcYOWLeteGxwzOtXjdRJy0yWRy_0qLDo6mMONSyWoPs6gtebITpWVimkpiBx_XXEKXkVTcIEJXIUFmkTzZtlD9wwKmCueopfWhdpc7KMiK8QsRhNcHArRyP3536CLUeKPlkAczCI2ZxF47eVuwQXX-YCekOkvx0ZMqHLDKu-Xh8usMxg3WAt2eI3TjqNZv0iv-Duvhg-JhfjSgZnbajogoNkeKdZfeLDz4hBMgE1CBxTtntfspldqq7QWjlEbQm13ejwzGk5zGCJyKbKvNN8vPYzxubT6rC7OazvVJZmQcq6l92BOMsVasIV_g2ecsPqSniDRg-zVjySrLv8xPdo1WjpeBnNueLi_YjUUt9RJ7bzjIrmHNt9PXhRiBy0"/>

Lets import SDK

In [17]:
const { createWallet } = require('@affinidi/wallet-node-sdk')
const walletFactory = createWallet('AffinityCore')

Lets setup wallets

In [21]:
const accessApiKey = '<yourkey>'

In [23]:
const options = {
    env: 'prod',
    accessApiKey:accessApiKey,
}
  const verifierWallet = await walletFactory.createWallet(options, 'P@55word!!!')
  const holderWallet = await walletFactory.createWallet(options, 'P@55word!!!')

## Verifier create Presentation Challenge

In [44]:
   const presentationChallenge = await verifierWallet.generatePresentationChallenge([
      { type: ['VerifiableCredential', 'ContentLike'] },
    ])

In [45]:
JSON.stringify(jwt_decode(presentationChallenge),null,2)

{
  "interactionToken": {
    "credentialRequirements": [
      {
        "type": [
          "VerifiableCredential",
          "ContentLike"
        ],
        "constraints": []
      }
    ],
    "callbackURL": ""
  },
  "exp": 1663398268356,
  "typ": "credentialRequest",
  "jti": "038e7a1d81ae3d05",
  "iss": "did:elem:EiBsQ5S0kNYU-CkVLHFVQQxSp58gpcQP9zg1nFGTr-avjQ;elem:initial-state=eyJwcm90ZWN0ZWQiOiJleUp2Y0dWeVlYUnBiMjRpT2lKamNtVmhkR1VpTENKcmFXUWlPaUlqY0hKcGJXRnllU0lzSW1Gc1p5STZJa1ZUTWpVMlN5SjkiLCJwYXlsb2FkIjoiZXlKQVkyOXVkR1Y0ZENJNkltaDBkSEJ6T2k4dmR6TnBaQzV2Y21jdmMyVmpkWEpwZEhrdmRqSWlMQ0p3ZFdKc2FXTkxaWGtpT2x0N0ltbGtJam9pSTNCeWFXMWhjbmtpTENKMWMyRm5aU0k2SW5OcFoyNXBibWNpTENKMGVYQmxJam9pVTJWamNESTFObXN4Vm1WeWFXWnBZMkYwYVc5dVMyVjVNakF4T0NJc0luQjFZbXhwWTB0bGVVaGxlQ0k2SWpBeVptVmxZbU16WkdJNE1UYzJPRGc0TXpRek9EWmxOVFJqTkRKalpqTmtOMll3TjJFMU1UWTVOVEJqWm1NNFpUYzJPRGcxTTJZek56a3dNbVkwWlRZNE1TSjlMSHNpYVdRaU9pSWpjbVZqYjNabGNua2lMQ0oxYzJGblpTSTZJbkpsWTI5MlpYSjVJaXdpZEhsd1pTSTZJbE5sWTNBeU5UWnJNVlp

## Lest prepare credential

In [46]:
const jsonSchema = 'https://schema.affinidi.com/ContentLikeV1-0.json'
const jsonContext = 'https://schema.affinidi.com/ContentLikeV1-0.jsonld'
const id = `claimId:${(Math.random() + 1).toString(36).substring(2)}`
const unsignedVC = {
    '@context': ['https://www.w3.org/2018/credentials/v1', jsonContext],
    id,
    type: ['VerifiableCredential', 'ContentLike'],
    holder: {
      id: holderWallet.did
    },
    credentialSubject: {
      data: {
        url: 'https://www.youtube.com/watch?v=owbkzvLhblk',
        date: new Date().toISOString(),
        like: true,
        score: 10
      },
    },
    credentialSchema: {
      id:  jsonSchema,
      type: 'JsonSchemaValidator2018',
    },
    issuanceDate: new Date().toISOString(),
    expirationDate: '2065-09-10T00:00:00.000Z',
  }   
const signedCredential = await holderWallet.signUnsignedCredential(unsignedVC)


## Create Verifiable Presentation

In [47]:
const vp = await holderWallet.createPresentationFromChallenge(
      presentationChallenge,
      [signedCredential],
      'youtube.com',
    )
JSON.stringify(vp,null,2)

{
  "@context": [
    "https://www.w3.org/2018/credentials/v1"
  ],
  "id": "presentationId:f461e8a2b035d689",
  "type": [
    "VerifiablePresentation"
  ],
  "holder": {
    "id": "did:elem:EiC5XYylCfH47s7dcJt2SpdBg1yS2lyuJpR6J8WAY0swRw;elem:initial-state=eyJwcm90ZWN0ZWQiOiJleUp2Y0dWeVlYUnBiMjRpT2lKamNtVmhkR1VpTENKcmFXUWlPaUlqY0hKcGJXRnllU0lzSW1Gc1p5STZJa1ZUTWpVMlN5SjkiLCJwYXlsb2FkIjoiZXlKQVkyOXVkR1Y0ZENJNkltaDBkSEJ6T2k4dmR6TnBaQzV2Y21jdmMyVmpkWEpwZEhrdmRqSWlMQ0p3ZFdKc2FXTkxaWGtpT2x0N0ltbGtJam9pSTNCeWFXMWhjbmtpTENKMWMyRm5aU0k2SW5OcFoyNXBibWNpTENKMGVYQmxJam9pVTJWamNESTFObXN4Vm1WeWFXWnBZMkYwYVc5dVMyVjVNakF4T0NJc0luQjFZbXhwWTB0bGVVaGxlQ0k2SWpBelpXRTJZamsyT1dabU1qZG1ZelppWlRKaE56TTFNREExTW1FMFpqWTNaamt4WWpjelltRmpNemMxT0RCak9UQTJabVUxTnpWbFpEVXpPVEpsTURWaVlTSjlMSHNpYVdRaU9pSWpjbVZqYjNabGNua2lMQ0oxYzJGblpTSTZJbkpsWTI5MlpYSjVJaXdpZEhsd1pTSTZJbE5sWTNBeU5UWnJNVlpsY21sbWFXTmhkR2x2Ymt0bGVUSXdNVGdpTENKd2RXSnNhV05MWlhsSVpYZ2lPaUl3TWpSaU9Ea3pNRFJoWmpObE9XTTNaRGxtT0RRNU5qTTJZalUyWldGa056azRNalkyWld

## Verification

In [48]:
   const result = await verifierWallet.verifyPresentation(vp)
  result

{
  isValid: true,
  did: 'did:elem:EiC5XYylCfH47s7dcJt2SpdBg1yS2lyuJpR6J8WAY0swRw;elem:initial-state=eyJwcm90ZWN0ZWQiOiJleUp2Y0dWeVlYUnBiMjRpT2lKamNtVmhkR1VpTENKcmFXUWlPaUlqY0hKcGJXRnllU0lzSW1Gc1p5STZJa1ZUTWpVMlN5SjkiLCJwYXlsb2FkIjoiZXlKQVkyOXVkR1Y0ZENJNkltaDBkSEJ6T2k4dmR6TnBaQzV2Y21jdmMyVmpkWEpwZEhrdmRqSWlMQ0p3ZFdKc2FXTkxaWGtpT2x0N0ltbGtJam9pSTNCeWFXMWhjbmtpTENKMWMyRm5aU0k2SW5OcFoyNXBibWNpTENKMGVYQmxJam9pVTJWamNESTFObXN4Vm1WeWFXWnBZMkYwYVc5dVMyVjVNakF4T0NJc0luQjFZbXhwWTB0bGVVaGxlQ0k2SWpBelpXRTJZamsyT1dabU1qZG1ZelppWlRKaE56TTFNREExTW1FMFpqWTNaamt4WWpjelltRmpNemMxT0RCak9UQTJabVUxTnpWbFpEVXpPVEpsTURWaVlTSjlMSHNpYVdRaU9pSWpjbVZqYjNabGNua2lMQ0oxYzJGblpTSTZJbkpsWTI5MlpYSjVJaXdpZEhsd1pTSTZJbE5sWTNBeU5UWnJNVlpsY21sbWFXTmhkR2x2Ymt0bGVUSXdNVGdpTENKd2RXSnNhV05MWlhsSVpYZ2lPaUl3TWpSaU9Ea3pNRFJoWmpObE9XTTNaRGxtT0RRNU5qTTJZalUyWldGa056azRNalkyWldNMFlqWTNOR0pqTlRZNE1qVXpZalF6WVdRNE9UVTVNRGt6TURZaWZWMHNJbUYxZEdobGJuUnBZMkYwYVc5dUlqcGJJaU53Y21sdFlYSjVJbDBzSW1GemMyVnlkR2x2YmsxbGRHaHZaQ0k2V3lJamNISnBiV0